# EDA for the Ninapro DB 8

In [1]:
import h5py
import os
import scipy.io
from collections import Counter
import matplotlib.pyplot as plt
import logging
import matplotlib.gridspec as gridspec
import pandas as pd
import seaborn as sns
from scipy import signal
from tqdm import tqdm
from scipy.fft import fft, fftfreq, rfft, rfftfreq
import numpy as np

pd.set_option('display.max_rows', 1000)



In [2]:
COLOR_DICT = {'midnight_blue': '#2c3e50', 'pomgrenate': '#c0392b', 'pumpkin': '#d35400', 
              'green_sea': '#16a085',
              'wisteria': '#8e44ad', 'orange': '#f39c12', 'turquoise': '#1abc9c', 'grey': '#7f8c8d',
             'lotus':'#f368e0', 'nasu':'#5f27cd'}


In [3]:
def parse_mat_file_to_dict(filepath):
    data = {}
    f = h5py.File(filepath, 'r')
    for k, v in f.items():
        data[k] = np.array(v)
    return f, data

# Loading Dataset

In [4]:
data_dir = 'data/'
subject = 1
subject_dataset = 1
subject_file = f'S{subject}_E1_A{subject_dataset}.mat'  #S1_E1_A1.mat
mat = scipy.io.loadmat(os.path.join(data_dir,subject_file))

In [5]:
mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Thu Dec 06 15:23:01 2018',
 '__version__': '1.0',
 '__globals__': [],
 'subject': array([[1]], dtype=uint8),
 'exercise': array([[1]], dtype=uint8),
 'emg': array([[-1.2213888e-05,  1.2848358e-05,  1.4171899e-06, ...,
          2.4547974e-06, -3.5635583e-07,  3.9899796e-06],
        [-1.1034591e-05,  1.1959419e-05, -8.0825924e-07, ...,
          7.5772382e-06, -7.6994631e-08,  9.7154589e-06],
        [-5.5595945e-07,  1.0261851e-05, -9.0351966e-07, ...,
          9.7515685e-06, -2.5459797e-06,  6.3704433e-06],
        ...,
        [-3.5108628e-06, -1.2804191e-05,  1.3349400e-06, ...,
         -3.4621539e-07, -2.2962845e-06,  1.2677652e-06],
        [-9.1614737e-07, -1.4396191e-05,  3.9385824e-07, ...,
         -1.8846814e-06, -2.8121669e-06,  1.2581410e-07],
        [ 8.8349225e-08, -1.3151894e-05, -2.0248261e-07, ...,
         -2.3238319e-06, -3.3372892e-06,  1.1472472e-07]], dtype=float32),
 'acc': array([[ 0.967795

# Dataset Description

### Acquisition
For each participant, three datasets were collected: 
- the first two datasets (acquisitions 1 & 2) comprised **10 repetitions of each movement** and the third dataset (acquisition 3) comprised only two repetitions. 

The variables included in the .mat files are the following:
- subject: subject number
- exercise: exercise number (value set to 1 in all data files)
- emg (16 columns): sEMG signals from the 16 sensors
- acc (48 columns): three-axis accelerometer data from the 16 sensors
- gyro (48 columns): three-axis gyroscope data from the 16 sensors
- mag (48 columns): three-axis magnetometer data from the 16 sensors
- glove (18 columns): calibrated signals from the 18 sensors of the Cyberglove
- stimulus (1 column): the movement repeated by the subject
- restimulus (1 column): again the movement repeated by the subject. In this case, the duration of the movement label is refined a-posteriori in order to correspond to the real movement.
- repetition (1 column): repetition number of the stimulus
- rerepetition (1 column): repetition number of restimulus


### Sampling frequency and Preprocessing
- The muscular activity was recorded using 16 active double-differential wireless sensors from a Delsys Trigno IM Wireless EMG system. The sensors comprise EMG electrodes and 9-axis inertial measurement units (IMUs). The sensors were positioned in two rows of eight units around the participants’ right forearm in correspondence to the radiohumeral joint (see pictures below). No specific muscles were targeted. The sensors were fixed on the forearm using the standard manufacturer-provided adhesive bands. 
- Moreover, a hypoallergenic elastic latex-free band was placed around the sensors to keep them fixed during the acquisition. 
- Sampling rate:
    - **The sEMG signals were sampled at a rate of 1111 Hz**, 
    - **accelerometer and gyroscope data were sampled at 148 Hz**
    - **magnetometer data were sampled at 74 Hz**. 
    - **All signals were upsampled to 2 kHz and post-synchronized.**
    
### Recorded target (i.e hand movements)
- During the acquisition, the subjects were asked to repeat 9 movements using both hands (bilateral mirrored movements). 
- The duration of each of the nine movements varied between **6 and 9 seconds** and consecutive trials were interleaved with **3 seconds of rest**. 
- Each repetition started with the participant holding their fingers at the rest state and involved slowly reaching the target posture as shown on the screen and returning to the rest state before the end of the trial. 

- The following movements were included:
    0. rest
    1. thumb flexion/extension
    2. thumb abduction/adduction
    3. index finger flexion/extension
    4. middle finger flexion/extension
    5. combined ring and little fingers flexion/extension
    6. index pointer
    7. cylindrical grip
    8. lateral grip
    9. tripod grip



# Dataset Insights

In [6]:
print(f"EMG data:{mat['emg'].shape}\nGlove data:{mat['glove'].shape} \nacc data:{mat['acc'].shape}")
print(f"\nstimulus data:{mat['stimulus'].shape} \nrestimulus data:{mat['restimulus'].shape}")
print(f"\nrepetition data:{mat['repetition'].shape} \nrerepetition data:{mat['rerepetition'].shape}")


print(f"Rerepetition value count:\n{Counter(mat['rerepetition'][:,0])}")
Counter(mat['restimulus'][:,0])

EMG data:(2292526, 16)
Glove data:(2292526, 18) 
acc data:(2292526, 48)

stimulus data:(2292526, 1) 
restimulus data:(2292526, 1)

repetition data:(2292526, 1) 
rerepetition data:(2292526, 1)
Rerepetition value count:
Counter({1: 291411, 6: 235135, 4: 231914, 8: 225229, 10: 224936, 2: 219693, 3: 216540, 5: 215265, 9: 213469, 7: 209056, 0: 9878})


Counter({0: 1188175,
         1: 82564,
         2: 93842,
         3: 160468,
         4: 120850,
         5: 112737,
         6: 88592,
         7: 175325,
         8: 120336,
         9: 149637})

- sEMG has 16 channels.
- 'rerepetition' variable defines the number of trial per movement.
- 'restimulus' defines the target gesture to be performed by subject. This variable takes in a value from 0 to 9 representing the 10 classes of gestures + rest

In [7]:
Counter(mat['stimulus'][:,0])

Counter({0: 796495,
         1: 139161,
         2: 141287,
         3: 168908,
         4: 147710,
         5: 147201,
         6: 147253,
         7: 218899,
         8: 181869,
         9: 203743})

In [8]:
FS = 2000
N_CH = mat['emg'].shape[1]

rec_duration = mat['emg'].shape[0]/FS
rec_time = np.arange(0,rec_duration, 1/FS)
print(f"Total recording duration for this subject {subject}: {rec_duration} seconds")
print(f"Creating time axis: \nbeg {rec_time[:5]} \nend {rec_time[-5:]}\n")
print(f"sEMG Channels: {N_CH}")

NameError: name 'np' is not defined

In [ ]:
# loading into df for easier handling
emg_df = pd.DataFrame(mat['emg'])
emg_df['time'] = rec_time
emg_df['restimulus'] = mat['restimulus']
emg_df['rerepetition'] = mat['rerepetition']
emg_df['trial'] = (emg_df['restimulus'] != emg_df['restimulus'].shift()).cumsum()

emg_df

## EMG Summary statistics

In [ ]:
df = emg_df.iloc[:,:N_CH]  # shorthand for extracting only channel columns
df.describe()

In [ ]:
ch_mean = df.mean()
ch_std = df.std()


fig = plt.figure(figsize=(15,5))
gs = gridspec.GridSpec(ncols=2, nrows=1)
ax = fig.add_subplot(gs[0])

df.mean().plot(kind='bar', legend=True, ax= ax, title="Mean EMG Amplitudes")
plt.xlabel('Channels')

ax = fig.add_subplot(gs[1])
ch_mean.plot(yerr=ch_std, kind='bar', title='Std EMG Amplitudes')
plt.xlabel('Channels');

# ch_mean.plot(kind = "barh", y = "mean", legend = False,
#         title = "Average Prices", xerr = "std")

There is a high variance across the channels but also for a single channel between the different gestures

### Gestures Duration

In [ ]:
# The first time a new gesture is introduced: i.e time for all trials per gesture
# Adjust the last restimulus duration
gesture_first_time = emg_df.groupby('restimulus').first()['time'] 
gestures_duration = gesture_first_time.diff().shift(-1)
gestures_duration[9] = emg_df.groupby('restimulus').last()['time'][9] - emg_df.groupby('restimulus').first()['time'][9]

fig = plt.figure(figsize=(20,5))
gs = gridspec.GridSpec(nrows=1, ncols=3)
ax = fig.add_subplot(gs[0])
gesture_first_time.plot(kind='barh', title="Start time of first repetition per gesture", ax= ax)


ax = fig.add_subplot(gs[1])
gestures_duration.plot(kind='bar', title="Duration of each gesture (all repetitions inc. Rest )", ax= ax,
                      ylabel='Time [sec]', xlabel='restimulus')


print(f'Duration of each gesture\n{gestures_duration}')

ax = fig.add_subplot(gs[2])
exact_stim_duration = emg_df.groupby(['restimulus','trial'])['time'].last() - emg_df.groupby(['restimulus','trial'])['time'].first()
exact_stim_duration.groupby('restimulus').mean().plot.bar(title='Avg. Duration per stimulus', ylabel='Time [sec]');

**Interpretation**
- Rest (0) is the first movement, it takes 9.947 seconds then all repetitions of gesture 1 are performed which take 113.44 seconds and so on.
- **Note** that Rest trials are interleaved between the gestures, so the duration of rest here is that of the first repetition only. Also the reported duration for gestures is that of all repetitions including the rest trials in between the repetitions.

### Mean EMG per gesture

In [ ]:
fig = plt.figure(figsize=(20,9))
ax  = fig.add_subplot()
emg_df.groupby('restimulus')[np.arange(N_CH)].mean().T.plot.bar(ax=ax, ylabel='Mean amplitudes',
                                                               xlabel='Channels',
                                                               color=list(COLOR_DICT.values()),
                                                               linewidth=1,
                                                               width=0.72, rot = 0)

ax.tick_params(axis='both', labelsize=14)
ax.set_ylabel(ylabel='Mean amplitudes',fontsize=18)
ax.set_xlabel(xlabel='Channels',fontsize=18)
ax.legend(ncol=5)
fig.suptitle('Mean EMG amplitude per stimulus', fontsize=20);


In [ ]:
# mean ampltitude per stimulus (should get the same plot as above)
# test = (emg_df.groupby(['restimulus','trial']).mean()).groupby('restimulus')[np.arange(N_CH)].mean()
# test.T.plot.bar()

- Channels are selective of the stimulus. For example channel 8 captures more stimuli 2,4,5,8 and 7.


In [ ]:
# correlation per stimulus
emg_gb_stim = emg_df.groupby('restimulus')[np.arange(N_CH)].mean()
cm = emg_gb_stim.corr()
plt.figure(figsize=(18,9))
sns.heatmap(cm, annot=True, cmap = 'viridis')
plt.title("Channel-wise correlations using mean amplitudes (across all stimuli)", fontsize=15);
plt.xlabel('Channels', fontsize=15)
plt.ylabel('Channels',fontsize=15);

In [ ]:
# correlation between stimulus using mean amplitudes
emg_gb_stim = emg_df.groupby('restimulus')[np.arange(N_CH)].mean().T
stimulus_name = {0: 'rest', 1: 'thumb flex/ext',2: 'thumb ab/ad',
                 3:'index flex/ext' ,
                 4:'mid. flex/ext', 5:'r&l flex/ext',
                 6:'index pointer', 7:'cyl. grip', 8:'lateral grip', 9:'tripod grip'}
cm = emg_gb_stim.corr()
plt.figure(figsize=(18,9))
sns.heatmap(cm, annot=True,xticklabels=stimulus_name.values(), yticklabels=stimulus_name.values())
# plt.title("Correlation between the channels mean amplitudes (across all stimuli)", fontsize=15);
plt.xlabel('Stimulus', fontsize=15)
plt.ylabel('Stimulus',fontsize=15);

In [ ]:
# cm.sort_values(by=[7])

In [ ]:
# quick test
stim_pair = [9,1]
stim_0_test = emg_df[emg_df['restimulus']==stim_pair[0]][np.arange(N_CH)].mean(axis=0)
stim_1_test = emg_df[emg_df['restimulus']==stim_pair[1]][np.arange(N_CH)].mean(axis=0)


np.corrcoef(stim_0_test, stim_1_test)

To compute the stimulus-wise correlation:
- Get the mean of the channels for each stimulus then compute the correlation between the stimuli.

**Observe the following**
1. High correlation between tripod grip and both thumb flexion/extension and index flex/ext. This makes sense since the same fingers are flexed to perform this grip.
2. Surprisingly, rest is also correlated with the grasps.


### Signal Visualization

In [ ]:
# From a single channel

ch = 0

fig = plt.figure(figsize=(20,5))
gs =gridspec.GridSpec(ncols=2, nrows=1)
ax = fig.add_subplot(gs[0])

ax.plot(rec_time, emg_df[ch],color=COLOR_DICT['midnight_blue'], label=f'Ch{ch}')
        
# plt.title('Single channel EMG [entire recording]')
plt.legend()
plt.xlabel('time [s]');
plt.ylabel('amplitude');

ax = fig.add_subplot(gs[1])
sns.lineplot(data=emg_df, x="time", y=0, hue="restimulus",
             palette=sns.color_palette(list(COLOR_DICT.values())))
fig.suptitle('Single channel EMG [entire recording]');

In [ ]:
# Zooming in
ch = 0
subset_N  = 300000
plt.figure(figsize=(15,9))
sns.lineplot(data=emg_df.loc[:subset_N], x="time", y=ch, hue="restimulus", 
             palette=sns.color_palette(list(COLOR_DICT.values()))[:3])
plt.xticks(ticks=np.arange(0,150,5));
plt.grid(True)
plt.title(f"Channel {ch}")

In [ ]:
# single finger movement
finger = 'index flex/ext'
index_finger_id = list(stimulus_name.keys())[list(stimulus_name.values()).index(finger)]
rep = 2

single_df = emg_df[(emg_df['restimulus']==index_finger_id) & (emg_df['rerepetition'] == rep)]
ch = 0
subset_N  = 300000
plt.figure(figsize=(15,9))
sns.lineplot(data=single_df, x="time", y=ch, hue="restimulus", 
             palette=sns.color_palette(list(COLOR_DICT.values()))[:1])
plt.grid(True)
plt.title(f"EMG Channel {ch} Repetition#{rep} for {finger}");


## Glove data

In [ ]:
glove_df = pd.DataFrame(mat['glove'])
glove_df['time'] = rec_time
glove_df['restimulus']= mat['restimulus']
glove_df['rerepetition']= mat['rerepetition']
glove_df['trial'] = emg_df['trial']

print(f"Checking on unique restimulus {glove_df[glove_df['rerepetition']==2]['restimulus'].unique()}")
print(f"Checking on id of first repetition per restimulus:\n {glove_df.groupby('restimulus')['rerepetition'].first()}")

### Tracking Index Finger Marker

In [ ]:
glove_df[(glove_df['restimulus']==1) & (glove_df['trial'] == 2)]  
    
glove_df.groupby(['restimulus','trial']).first()

In [ ]:
# manually picking the trial nuumber that would correspond to flex/extend finger
finger = 'index flex/ext'
index_finger_id = list(stimulus_name.keys())[list(stimulus_name.values()).index(finger)]
rep = 42
single_finger_trial = glove_df[(glove_df['restimulus']==index_finger_id) & (glove_df['trial'] == rep)]  

glove_index_ch = 5  # channel 6 is on the index finger joint
subset_N  = 300000
plt.figure(figsize=(15,5))
sns.lineplot(data=single_finger_trial, x="time", y=glove_index_ch, hue="restimulus", 
             palette=sns.color_palette(list(COLOR_DICT.values()))[:1])
plt.grid(True)
plt.title(f"Single trial (#{rep}) Glove data for {finger} - Channel# {glove_index_ch}", fontsize=16)
plt.xlabel('time [sec]')


fig = plt.figure(figsize=(15,5))
gs = gridspec.GridSpec(nrows=1, ncols=1)
ax = fig.add_subplot(gs[0])

for rep in range(1,10):
    index_trial = glove_df[(glove_df['restimulus']==index_finger_id) & (glove_df['rerepetition'] ==rep)][glove_index_ch]
    index_trial.plot(label=f"Trial#{rep}", ax= ax)
plt.legend(ncol=5)
plt.ylabel('Position [a.u.]')
plt.xlabel('time [sec]')
plt.title('')


In [ ]:
# fig = plt.figure(figsize=(20,5))
# gs = gridspec.GridSpec(nrows=2, ncols=5)

# for i, rep in enumerate(range(1,10)):
#     ax = fig.add_subplot(gs[i])

#     index_trial = glove_df[(glove_df['restimulus']==index_finger_id) & (glove_df['rerepetition'] ==rep)][glove_index_ch]
#     index_trial.plot(title=f"Trial {rep}", ax= ax)
#     plt.xlabel('Time [sec]')


In [ ]:
# Checking on a different channel
fig = plt.figure(figsize=(15,5))
gs = gridspec.GridSpec(nrows=1, ncols=1)
ax = fig.add_subplot(gs[0])
glove_ch = 4
for rep in range(1,10):
    index_trial = glove_df[(glove_df['restimulus']==index_finger_id) & (glove_df['rerepetition'] ==rep)][glove_ch]
    ax.plot(index_trial, label=f"Trial#{rep}")
plt.legend(ncol=5)
plt.ylabel('Position [a.u.]')
plt.xlabel('time [sec]')
plt.title(f'Glove Channel#{glove_ch}',fontsize=14)



# Is the date preprocessed already?

<mark>**Note**: to check if the data is already pre-processed, I can check on the Fourier Transform of the signal and see if the signal is attenuated at frequencies multiple of 50 Hz (power-line interferance)</mark>


## Conventional preprocessing
Usually to filter the data we follow the classic:
1. Notch filters for the power line interference
2. Band pass filters


For example, in [Hyser Paper](https://ieeexplore.ieee.org/document/9438637), a typical pre-processing pipleine looks like:

> "The acquired HD-sEMG signals were first filtered with **10 Hz highpass and 500 Hz lowpass Butterworth filters** (both are zero-phase digital filters processing the signals in both the forward and reverse directions, 8-order each direction). **A notch filter combination was then used to attenuate the power line interference at 50 Hz and all harmonic components up to 400 Hz**."




In [ ]:
# Plot FFT
fig = plt.figure(figsize=(15,15))
gs = gridspec.GridSpec(3, 1)
N = emg_df.shape[0]

ax = fig.add_subplot(gs[0])
yf = fft( np.array(emg_df[0]))
xf = fftfreq(N, 1 / FS)
ax.plot(xf, np.abs(yf))
plt.xlabel('freq[Hz]')
plt.ylabel('FFT Amplitude |X(freq)|')
plt.xlim([-200,200])
plt.show()

**Note**: there is already a notch filter (I think) applied to the signal. Frequencies at multiples of 50 Hz are already attenuated.

In [ ]:
def create_notch_filters(pipeline, samp_freq):
    w0 = pipeline['notch_reject']
    Q = pipeline['notch_reject'] / pipeline['notch_bandwidth']
    fs = samp_freq
    coeff_tuples = [(signal.iirnotch(w0 * i, Q, fs)) for i in range(1, 11)]
    return coeff_tuples


def create_bp_filter(prepipeline, samp_freq):
    bp_order, bp_cutoff_freq = prepipeline['bp_order'], prepipeline['bp_cutoff_freq']
    b_bp, a_bp = signal.butter(bp_order, bp_cutoff_freq / (samp_freq / 2), btype='bandpass')
    return b_bp, a_bp

def filter_data(data, b_coeff, a_coeff):
    return signal.filtfilt(b_coeff, a_coeff, data)

In [ ]:
# pipeline = {'notch_reject': 50, 'notch_bandwidth': 0.5,
#            'bp_order':2 , 'bp_cutoff_freq':np.array([2, 500]) }